In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time
import requests
import os
import csv
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from pathlib import Path
from datetime import datetime
import sqlite3

In [2]:
from splinter import Browser
from selenium.webdriver.chrome.service import Service

my_service = Service(executable_path=r'C:\Users\pc\OneDrive\Desktop\TCG-scraper\chromedriver.exe')
browser = Browser('chrome', service=my_service)

In [3]:

sset = ["innistrad-crimson-vow", "ixalan", "kaldheim", "innistrad-midnight-hunt", "war-of-the-spark", "kamigawa-neon-dynasty"]
#sset = ['ixalan']

In [4]:
for x in sset:
    url =f"https://shop.tcgplayer.com/price-guide/magic/{x}"
    browser.visit(url)
    browser.driver.maximize_window()
    time.sleep(7)
    req = browser.html
    soup = BeautifulSoup(req, "html")
    names= []
    title_tags = soup.find_all('a', class_ = "pdp-url")
    for i in title_tags:
       names.append(i.text)
    prices =[]
    price = soup.find_all('td', class_ = "tcg-table-body__cell tcg-table-body__cell--align-right")
    count = -1
    for i in price:
        count = count +1
        if  count%2 > 0 :
            prices.append(i.text)
    price_list= zip(names, prices)
    mkt_df = pd.DataFrame(price_list,) 
    mkt_df = mkt_df.reset_index(drop=True)
    mkt_df.columns = ['Name', 'Market Price']
    mkt_df['date']= datetime.today().strftime('%Y-%m-%d')
    
    mkt_df['Unnamed: 0']= 2
    mkt_df = mkt_df[['Unnamed: 0', 'Name', 'Market Price', 'date']]
    price =  Path("../Resources/price.db")
    con = sqlite3.connect(price)
    cur = con.cursor()
    d = datetime.today().strftime('%Y-%m-%d')
    cur.execute(f"select * from '{x}_priceData' where date = '{d}'")
    if cur.fetchone():
        print(f"duplicate prvented for {x}")
    else:
        for row in mkt_df.itertuples():
            insert_sql = f''' INSERT INTO '{x}_priceData' ('Unnamed: 0', Name, 'Market Price', date) VALUES ('{row[1]}',"{row[2]}",'{row[3]}','{row[4]}')'''
            cur.execute(insert_sql)
        print(f'{x} has been update')
    #con.commit()
    con.close()

innistrad-crimson-vow has been update
duplicate prvented for ixalan
kaldheim has been update
duplicate prvented for innistrad-midnight-hunt
war-of-the-spark has been update
duplicate prvented for kamigawa-neon-dynasty


In [5]:
mkt_df.to_csv(f'{x}_priceData.csv')